# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

# Set Up the Call Spread Strategy
Welcome to this notebook in which you will learn to set up a popular strategy called the bull call spread strategy. A bull call spread is an options trading strategy that involves purchasing call options with a lower strike price and selling the same number of call options with a higher strike price. The goal of the strategy is to profit from the difference between the two strike prices, with limited risk.

The notebook is structured as follows:
1. [Import Libraries](#libraries)
1. [Import the Data](#import)
2. [Get the ATM Strike Prices](#atm_strikes)
4. [Set Up the ATM Options](#atm_options)
5. [Populate the Premium of the Options](#premium)
6. [How to Set Up a Bull Call Spread Strategy?](#spread)
7. [Conclusion](#conclusion)

<a id='libraries'></a>
## Import Libraries
We import the necessary libraries to make our analysis.

In [1]:
# For data manipulation
import pandas as pd
import numpy as np

# To ignore warning statements
import warnings
warnings.filterwarnings('ignore')

<a id='import'></a>
## Import the Data
In order to create the bull call spread strategy, we need the options data and its underlying asset price data. We import the options chain data from the pickle file `spx_eom_expiry_options_2010_2022.bz2`. 

In [2]:
# Import the option chain data for S&P 500
options_data = pd.read_pickle(
    '../data_modules/spx_eom_expiry_options_2010_2022.bz2')

# Remove unnecessary square brackets and spaces from column names
options_data.columns = options_data.columns.str.replace(
    "[", "").str.replace("]", "").str.strip()

# Display the options data
options_data.tail()

,STRIKE,STRIKE_DISTANCE_PCT,C_LAST,UNDERLYING_LAST,P_LAST,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,P_DELTA,P_GAMMA,P_VEGA,P_THETA,P_RHO,P_IV
[QUOTE_DATE],,,,,,,,,,,,,,,,,,,
2022-09-30,6300.0,0.755,0.03,3589.7,2155.22,2022-09-30,0.0,0.00001,0.0,0.00073,-0.02515,-0.00046,3.467740,-0.81918,-0.00001,0.33024,-4.34966,-0.03739,8.585380
2022-09-30,6400.0,0.783,0.00,3589.7,0.00,2022-09-30,0.0,0.00000,0.0,0.00072,-0.02517,0.00048,3.560500,-0.81333,-0.00004,0.33582,-4.34954,-0.03792,8.895180
2022-09-30,6500.0,0.811,0.00,3589.7,0.00,2022-09-30,0.0,0.00000,0.0,0.00131,-0.02465,-0.00022,3.645170,-0.80773,0.00000,0.34053,-4.35038,-0.03851,9.204630
2022-09-30,6600.0,0.839,0.05,3589.7,0.00,2022-09-30,0.0,0.00004,0.0,0.00105,-0.02518,-0.00038,3.734590,-0.80204,0.00002,0.34486,-4.34980,-0.03926,9.514720
2022-09-30,6700.0,0.866,0.05,3589.7,0.00,2022-09-30,0.0,0.00000,0.0,0.00090,-0.02465,-0.00049,3.816150,-0.79602,0.00004,0.35040,-4.34985,-0.04006,9.824500


<a id='atm_strikes'></a>
## Get the ATM Strike Prices
First, let's pick a date to create the spread strategy. We will fetch the options and underlying asset data for the same date.

In [3]:
# Fetch data for analysis date
analysis_date = "2022-09-01"

# Select the underlying's close price data that corresponds to the date
underlying_price = options_data.loc[analysis_date, 'UNDERLYING_LAST'][0]

# Select the options data that corresponds to the date
options_chain = options_data.loc[analysis_date]

# Print the underlying's prices of the date
f"The underlying's price on {analysis_date} is {underlying_price}"

"The underlying's price on 2022-09-01 is 3968.05"

In [4]:
# Print the options data for the picked date
options_chain.head()

,STRIKE,STRIKE_DISTANCE_PCT,C_LAST,UNDERLYING_LAST,P_LAST,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,P_DELTA,P_GAMMA,P_VEGA,P_THETA,P_RHO,P_IV
[QUOTE_DATE],,,,,,,,,,,,,,,,,,,
2022-09-01,800.0,0.798,0.0,3968.05,0.00,2022-09-30,29.0,1.0,0.0,0.0,0.0,0.0,,-0.00039,0.0,0.00477,-0.01112,-0.00022,1.614410
2022-09-01,1000.0,0.748,0.0,3968.05,0.05,2022-09-30,29.0,1.0,0.0,0.0,0.0,0.0,,-0.00021,0.0,0.00520,-0.01150,-0.00091,1.392130
2022-09-01,1200.0,0.698,0.0,3968.05,0.07,2022-09-30,29.0,1.0,0.0,0.0,0.0,0.0,,-0.00011,0.0,0.00629,-0.01108,-0.00074,1.212150
2022-09-01,1300.0,0.672,0.0,3968.05,0.05,2022-09-30,29.0,1.0,0.0,0.0,0.0,0.0,,-0.00020,0.0,0.00624,-0.01135,-0.00080,1.132590
2022-09-01,1400.0,0.647,0.0,3968.05,0.07,2022-09-30,29.0,1.0,0.0,0.0,0.0,0.0,,-0.00025,0.0,0.00676,-0.01119,-0.00048,1.059260


Let's get the at-the-money option strike price that is closest to the underlying's close price of the above date. Since the SPX EOM option strike prices are given in multiples of 10, we do the following computation to get close to the strike price as per the underlying's price.

For example, if we have the underlying's close price as 3,589.7, we divide this number by 10 to get it rounded. 
```
multiplier = round(3589.7/10) 
multiplier = 359
```
Then, the strike multiple will be multiplied by the multiplier to get the closest option strike price as per the underlying's close price.
```
closest_strike_price = 10 * 359
closest_strike_price = 3590
```
This number will be the chosen strike price to go long on the ATM options. The above computation is done with just 2 code lines below. Finally, we print the ATM strike price that belongs to the picked date for analysis.

In [5]:
# Set the strike multiple
strike_multiple = 10

# Set the ATM strike price
atm_strike_price = strike_multiple * (round(underlying_price / strike_multiple))

# Print the ATM strike price of the date
f"The ATM strike price on {analysis_date} is {atm_strike_price}"

'The ATM strike price on 2022-09-01 is 3970'

<a id='atm_options'></a>
## Set Up the ATM Options
To create the bull call spread strategy, you need to go long on an ATM call option and short another call option at a higher strike price. 

Let's do the following:

1. Create a dataframe called `call_spread` to save its information.
2. Create the `Option Type` column.
3. Store the `Strike Price` for both option types
4. Set the value of the `position` column to 1 for long and -1 for short.

In [6]:
# Create the call_spread dataframe
call_spread = pd.DataFrame()

# Set up the first leg of the spread
call_spread['Option Type'] = ['CE', 'CE']

# Fill the strike price column with the ATM call's strike price
call_spread['Strike Price'] = [atm_strike_price, atm_strike_price+10]

# Set the position column to 1
call_spread['position'] = [1, -1]

# Print the call_spread dataframe
call_spread

,Option Type,Strike Price,position
0,CE,3970,1
1,CE,3980,-1


<a id='premium'></a>
## Populate the Premium of the Options
Let's add something important to our `call_spread` dataframe: the premium. We will fetch the premium of the ATM call option with a strike price of 3590. And later, we are going to create a function for that purpose. This is going to be useful for later computations. 

In [7]:
# Condition to specify the strike price of the call option
condition = options_chain.STRIKE == 3590

# Get the premium based on the above condition
premium = options_chain.loc[condition, 'C_LAST']

# Print the call premium
premium

 [QUOTE_DATE]
2022-09-01    301.8
Name: C_LAST, dtype: float64

For ease of use for future computations, let's create a function to get the same result as above. The structure of the function also follows what we did previously. The function will fetch either the `C.LAST` or the `P.LAST` values depending on whether the option is a call or a put respectively.

In [8]:
# Function to get the premium for an option contract
def get_premium(options_strategy, options_data):

    # Create to condition to assure we choose the correct option strike price
    strike = options_strategy['Strike Price']

    # Create to condition to assure we choose the correct option type
    option_type = options_strategy['Option Type']

    # Return the last price of the option that complies with the above conditions
    if option_type == 'CE':
        return options_data[options_data['STRIKE'] == strike].C_LAST
    if option_type == 'PE':
        return options_data[options_data['STRIKE'] == strike].P_LAST
    return 0

In [9]:
# Apply the function to the call_spread dataframe and store the values on the premium column
call_spread['premium'] = call_spread.apply(
    lambda r: get_premium(r, options_chain), axis=1)

# Print the updated call_spread dataframe
call_spread

,Option Type,Strike Price,position,premium
0,CE,3970,1,98.15
1,CE,3980,-1,102.00


<a id='spread'></a>
## How to Set Up a Bull Call Spread Strategy?

This notebook will stand out to you when you start trading because in this section we'll put together all that we did above into a function so that you can benefit from it. The only inputs you are going to need for the function is the options chain data. And finally, the function will return a dataframe output as you have seen previously.

Note that the difference between the OTM call strike price and the ATM call strike price here is calculated based on the 4 times the premium of the ATM call. However, you can opt to change this difference calculation based on your personal preference.

In [10]:
def setup_call_spread(options_data, strike_difference=10):
    
    # Create a dataframe
    call_spread = pd.DataFrame(columns=['Option Type', 'Strike Price', 'position', 'premium'])

    underlying_price = options_data['UNDERLYING_LAST'][0]

    # Calculate ATM strike price
    atm_strike_price = strike_difference * (round(underlying_price / strike_difference))

    # Set up first leg of the spread
    call_spread.loc['0'] = ['CE', atm_strike_price, 1, np.nan]
    
    # Append premium for the leg
    call_spread['premium'] = call_spread.apply(lambda r: get_premium(r, options_data), axis=1)
    
    # Define price deviation for next leg of spread
    deviation = round(call_spread.premium.sum()*4 / strike_difference) * strike_difference

    # Set up next leg of the spread
    call_spread.loc['1'] = ['CE', atm_strike_price + deviation, -1, np.nan]
    
    # Append respective premiums for the legs
    call_spread['premium'] = call_spread.apply(lambda r: get_premium(r, options_data), axis=1)

    return call_spread

In [11]:
# Get the call spread strategy dataframe
setup_call_spread(options_chain)

,Option Type,Strike Price,position,premium
0,CE,3970,1,98.15
1,CE,4360,-1,3.21


<a id='conclusion'></a>
## Conclusion

Now you know how to construct the bull call spread strategy. We have even created a function that includes all the necessary steps to get the strategy details in a dataframe as output. In the upcoming notebook, you will learn how to backtest this strategy.<br><br>